In [1]:
import google.datalab.bigquery as bq

In [5]:
%%bq query -n conversions
with
visits as

  (SELECT
      
       geoNetwork.region,
       device.deviceCategory as device,
       device.deviceCategory,
       sum(totals.visits) as sessions,
       sum(totals.totalTransactionRevenue) as revenue,
       sum(totals.transactions) as transactions

FROM
   `bigquery-public-data.google_analytics_sample.ga_sessions_*`
where  _TABLE_SUFFIX between '20170701' and '20170801' 
         and totals.visits is not null and
         geoNetwork.region not in ('not available in demo dataset')
group by 1,2,3)

select 
region,
(sum(transactions)/sum(sessions))*100 as conversion_rate

FROM visits 
GROUP BY region

having conversion_rate > 0
order by conversion_rate desc

In [6]:
%%bq execute -q conversions

region,conversion_rate
Wisconsin,14.2857142857
New Jersey,14.2857142857
Michigan,10.6918238994
Minnesota,10.5263157895
Georgia,9.24657534247
Massachusetts,9.22190201729
Oregon,8.64197530864
Illinois,8.5597826087
Colorado,8.33333333333
Washington,7.77502067825


In [8]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected= True)

In [9]:
df = conversions.execute(output_options=bq.QueryOutput.dataframe()).result()

In [10]:
data = dict(type= 'choropleth',
            locations = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MS','MN',
                        'MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC'],
            locationmode = 'USA-states',
            colorscale = 'viridis',
            reversescale = False,
            z= df['conversion_rate'],
            text = df['region'],
            colorbar = {'title':'conversion_rate'}
           )

In [11]:
layout = dict(title ='conversion 2018',geo = dict(scope='usa',projection=dict( type='albers usa' ),showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'))

In [12]:
choromap = go.Figure(data=[data],layout=layout)
iplot(choromap)